# Forecast with LSTM

## Introduction

In this notebook, I predict using long short-term memory (LSTM) models.  

I start with the San Juan data and forecast with 5 different "flavors" of LSTM.  

Next, I apply the most-successful approach to the Iquitos data.  

Finally, I collect my comments at the end.

## Set up

In [1]:
# Eliminate some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Import global libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns

# # Import converters to allow matplotlib to use dates and eliminte warnings
# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# Set some Seaborn defaults
sns.set_style('whitegrid')
sns.set_context('paper')
sns.set_palette('muted')

In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

# from keras.layers import Dropout
# from keras.layers import *
# from keras.callbacks import EarlyStopping


from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D

Using TensorFlow backend.


In [5]:
# Import homegrown functions
import my_func
import importlib  

## Load data

In [6]:
# Load whole datasets
train_and_test_sj = pd.read_pickle('../sb_cap2_nb-99_data/prep_sj.pickle')
train_and_test_iq = pd.read_pickle('../sb_cap2_nb-99_data/prep_iq.pickle')

# Load already-split datasets
train_sj = pd.read_pickle('../sb_cap2_nb-99_data/prep_train_sj.pickle')
test_sj = pd.read_pickle('../sb_cap2_nb-99_data/prep_test_sj.pickle')
train_iq = pd.read_pickle('../sb_cap2_nb-99_data/prep_train_iq.pickle')
test_iq = pd.read_pickle('../sb_cap2_nb-99_data/prep_test_iq.pickle')

# Load already split, log-transformed datasets
log_train_sj = pd.read_pickle('../sb_cap2_nb-99_data/prep_log_train_sj.pickle')
log_test_sj = pd.read_pickle('../sb_cap2_nb-99_data/prep_log_test_sj.pickle')
log_train_iq = pd.read_pickle('../sb_cap2_nb-99_data/prep_log_train_iq.pickle')
log_test_iq = pd.read_pickle('../sb_cap2_nb-99_data/prep_log_test_iq.pickle')

# Load scores
score_df = pd.read_pickle('../sb_cap2_nb-99_data/scores_after_nbk_8.pickle')

## Define functions

In [7]:
def LSTM_prep_raw_series_for_model_scaled(raw_seq, n_steps_in, n_steps_out):
    
    # Split up the series
    X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
    
    # Reshape into 3D of [samples, timestamps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    
    return X, y

In [8]:
def split_sequence(sequence, n_steps_in, n_steps_out):

    X, y = list(), list()
    for i in range(len(sequence)):
    
    # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        
        # check if we are beyond the sequence 
        if out_end_ix > len(sequence):
            break
       
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y)

In [9]:
def examine_the_shape_of_the_array(name, seq):
    
    print('{} has shape of: {}'.format(name, seq.shape))

In [10]:
def LSTM_prep_raw_series_for_forcast_scaled(raw_seq, n_steps_in, n_features):
    
    # Collect last observations of training set as input to forecast
    x_input = np.array(raw_seq)
    x_input = x_input[(n_steps_in*-1):]
    x_input = x_input.reshape((1, n_steps_in, n_features))

    return x_input

In [11]:
def combine_actual_and_LSTM_forecast_into_single_dataframe(actual, forecast):
    
    forecast_values = forecast.reshape(forecast.shape[1], forecast.shape[0])
    actual_df = actual.copy()
    actual_df['forecast'] = forecast_values
    actual_df.columns = ['actual', 'forecast']

    return actual_df

In [12]:
def graph_loss_by_epoch(data, model, city):
    """Visualize the declining loss acorss my 50 epochs"""
    
    g = plt.figure(figsize=(9, 3), dpi=100)
    g = plt.xlabel('Epochs')
    g = plt.ylabel('Loss')
    g = plt.title('Loss by epoch while fitting the {} model for {} data'.format(model, city))
    g = plt.xticks(np.arange(0,50,5))
    g = plt.plot(range(len(data)), data)

## Explore data

In [13]:
print('Train sj has a length of {}'.format(len(train_sj)))
print('Test sj has a length of {}'.format(len(test_sj)))
print('That is, I am looking for about a {} year forecast!'.format(int(len(test_sj)/52)))

Train sj has a length of 702
Test sj has a length of 234
That is, I am looking for about a 4 year forecast!


In [14]:
print('Train iq has a length of {}'.format(len(train_iq)))
print('Test iq has a length of {}'.format(len(test_iq)))
print('That is, I am looking for about a {} year forecast!'.format(int(len(test_iq)/52)))

Train iq has a length of 331
Test iq has a length of 111
That is, I am looking for about a 2 year forecast!


## Prep data

In [15]:
# Scale data - San Jose
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_sj)

prep_train_sj = scaler.transform(train_sj)
prep_train_sj = np.concatenate(prep_train_sj) # Flattens the array
prep_train_sj = prep_train_sj.tolist()

prep_test_sj = scaler.transform(test_sj)
prep_test_sj = np.concatenate(prep_test_sj) # Flattens the array
prep_test_sj = prep_test_sj.tolist()

In [16]:
# Scale data - log tranformed - San Jose
scaler_log = MinMaxScaler(feature_range=(0, 1))
scaler_log.fit(log_train_sj)

log_prep_train_sj = scaler_log.transform(log_train_sj)
log_prep_train_sj = np.concatenate(log_prep_train_sj) # Flattens the array
log_prep_train_sj = log_prep_train_sj.tolist()

log_prep_test_sj = scaler_log.transform(log_test_sj)
log_prep_test_sj = np.concatenate(log_prep_test_sj) # Flattens the array
log_prep_test_sj = log_prep_test_sj.tolist()

In [17]:
# Scale data - log tranformed - Iquitos
scaler_log_iq = MinMaxScaler(feature_range=(0, 1))
scaler_log_iq.fit(log_train_iq)

log_prep_train_iq = scaler_log_iq.transform(log_train_iq)
log_prep_train_iq = np.concatenate(log_prep_train_iq) # Flattens the array
log_prep_train_iq = log_prep_train_iq.tolist()

log_prep_test_iq = scaler_log_iq.transform(log_test_iq)
log_prep_test_iq = np.concatenate(log_prep_test_iq) # Flattens the array
log_prep_test_iq = log_prep_test_iq.tolist()

## Build models

### Variation 1 - LSTM - Simple

In [18]:
# Set up
this_run_train = prep_train_sj
this_run_test = prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'simple'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 1
this_run_transform = 'none'

In [19]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

In [20]:
# Prepare the time series for model
X, y = LSTM_prep_raw_series_for_model_scaled(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(LSTM(n_units, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = LSTM_prep_raw_series_for_forcast_scaled(raw_seq, n_steps_in, n_features)

In [ ]:
# Predict
yhat_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat = scaler.inverse_transform(yhat_scaled)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# # Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

### Variation 2 - LSTM - Stacked

In [ ]:
# Set up
this_run_train = prep_train_sj
this_run_test = prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'stacked'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 2
this_run_transform = 'none'

In [ ]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

In [ ]:
# Prepare the time series for model
X, y = LSTM_prep_raw_series_for_model_scaled(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(LSTM(n_units, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(n_units, activation='relu')) 
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = LSTM_prep_raw_series_for_forcast_scaled(raw_seq, n_steps_in, n_features)

In [ ]:
# Predict
yhat_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat = scaler.inverse_transform(yhat_scaled)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# # Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

### Variation 3 -  LSTM - Stacked - 2.5 times the nodes

In [ ]:
# Set up
this_run_train = prep_train_sj
this_run_test = prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'stacked, 2.5x nodes'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 3
this_run_transform = 'none'

In [ ]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 250 # Number of notes at each layer

In [ ]:
# Prepare the time series for model
X, y = LSTM_prep_raw_series_for_model_scaled(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(LSTM(n_units, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(n_units, activation='relu')) 
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = LSTM_prep_raw_series_for_forcast_scaled(raw_seq, n_steps_in, n_features)

In [ ]:
# Predict
yhat_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat = scaler.inverse_transform(yhat_scaled)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# # Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

### Variation 4 - LSTM - Bidirectional 

In [ ]:
# Set up
this_run_train = prep_train_sj
this_run_test = prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'bidirectional'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 4
this_run_transform = 'none'

In [ ]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

In [ ]:
# Prepare the time series for model
X, y = LSTM_prep_raw_series_for_model_scaled(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(Bidirectional(LSTM(n_units, activation='relu'), input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = LSTM_prep_raw_series_for_forcast_scaled(raw_seq, n_steps_in, n_features)

In [ ]:
# Predict
yhat_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat = scaler.inverse_transform(yhat_scaled)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# # Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

### Variation 5 - LSTM - CNN-LSTM

In [ ]:
# Set up
this_run_train = prep_train_sj
this_run_test = prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'CNN-LSTM'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 5
this_run_transform = 'none'

In [ ]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

n_seq = 2
n_steps2 = int(n_steps_in/n_seq)

In [ ]:
# Prepare the time series for CNN LSTM
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [ ]:
examine_the_shape_of_the_array('Shape of X after first split', X)

In [ ]:
# Reshape into 4D of [samples, subsequences, timesteps, features]
X = X.reshape((X.shape[0], n_seq, n_steps2, n_features))

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(TimeDistributed(Conv1D(64, 1, activation='relu'), input_shape=(None, n_steps2, n_features)))
model.add(TimeDistributed(MaxPooling1D()))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(n_units, activation='relu'))
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = np.array(raw_seq)
x_input = x_input[(n_steps_in*-1):]
x_input = x_input.reshape((1, n_seq, n_steps2, n_features))

In [ ]:
# Predict
yhat_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat = scaler.inverse_transform(yhat_scaled)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# # Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

## Variation 6 - ConvLSTM

In [ ]:
# Set up
this_run_train = prep_train_sj
this_run_test = prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'ConvLSTM'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 6
this_run_transform = 'none'

In [ ]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

n_seq = 2
n_steps2 = int(n_steps_in/n_seq)

In [ ]:
# Prepare the time series for CNN LSTM
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [ ]:
examine_the_shape_of_the_array('Shape of X after first split', X)

In [ ]:
# Reshape into 4D of [samples, subsequences, timesteps, features]
X = X.reshape((X.shape[0], n_seq, 1, n_steps2, n_features))

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(ConvLSTM2D(64, (1,2), activation='relu', input_shape=(n_seq, 1, n_steps2, n_features)))
model.add(Flatten())
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = np.array(raw_seq)
x_input = x_input[(n_steps_in*-1):]
x_input = x_input.reshape((1, n_seq, 1, n_steps2, n_features))

In [ ]:
# Predict
yhat_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat = scaler.inverse_transform(yhat_scaled)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

### Variation 7 - LSTM - ConvLSTM - log transformed data - San Juan

In [ ]:
# Set up
this_run_train = log_prep_train_sj
this_run_test = log_prep_test_sj
this_run_test_original = test_sj
this_run_approach = 'LSTM'
this_run_details = 'ConvLSTM'
this_run_city = 'san juan'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 7
this_run_transform = 'log(x+1)'

In [ ]:
# Define other parameters
n_steps_out = 234 # Number of timesteps as output.  Aka, forecast length
n_steps_in = n_steps_out * 2 # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

n_seq = 2
n_steps2 = int(n_steps_in/n_seq)

In [ ]:
# Prepare the time series for CNN LSTM
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# Reshape into 4D of [samples, subsequences, timesteps, features]
X = X.reshape((X.shape[0], n_seq, 1, n_steps2, n_features))

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(ConvLSTM2D(64, (1,2), activation='relu', input_shape=(n_seq, 1, n_steps2, n_features)))
model.add(Flatten())
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

# Graph loss
graph_loss_by_epoch(model.history.history['loss'], this_run_approach, str.title(this_run_city))

In [ ]:
# Prepare the last observations of time series as input to model
x_input = np.array(raw_seq)
x_input = x_input[(n_steps_in*-1):]
x_input = x_input.reshape((1, n_seq, 1, n_steps2, n_features))

In [ ]:
# Predict
yhat_log_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat_log = scaler_log.inverse_transform(yhat_log_scaled)

In [ ]:
# Invert log
yhat = np.expm1(yhat_log)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# # Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

## Summarize - all models from this notebook - San Juan

In [ ]:
score_df[(score_df['approach'] == 'LSTM') &  (score_df['city'] == 'san juan')].sort_values('mae')

## Variation 8 - LSTM - ConvLSTM - log transformed data - Iquitos

In [ ]:
# Set up
this_run_train = log_prep_train_iq
this_run_test = log_prep_test_iq
this_run_test_original = test_iq
this_run_approach = 'LSTM'
this_run_details = 'ConvLSTM'
this_run_city = 'iquitos'
this_run_data_set = 'test'

raw_seq = this_run_train

this_run_variation = 8
this_run_transform = 'log(x+1)'

In [ ]:
# Define other parameters
n_steps_out = 111 # Number of timesteps as output.  Aka, forecast length
n_steps_in = int(n_steps_out * 1.5) # Number of timesteps as input
n_features = 1 # Number of features.  Always 1 for this univariate forecast
n_units = 100 # Number of notes at each layer

n_seq = 2
n_steps2 = int(n_steps_in/n_seq)

In [ ]:
# Prepare the time series for CNN LSTM
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

In [ ]:
# Reshape into 4D of [samples, subsequences, timesteps, features]
X = X.reshape((X.shape[0], n_seq, 1, n_steps2, n_features))

In [ ]:
# Specify, compile, summarize and fit model

# Specify
model = Sequential()
model.add(ConvLSTM2D(64, (1,2), activation='relu', input_shape=(n_seq, 1, n_steps2, n_features)))
model.add(Flatten())
model.add(Dense(n_steps_out))

# Compile
model.compile(optimizer='adam', loss='mse')

# Summarize
model.summary()

# Fit
model.fit(X, y, epochs=50, verbose=0)

In [ ]:
# Prepare the last observations of time series as input to model
x_input = np.array(raw_seq)
x_input = x_input[(n_steps_in*-1):]
x_input = x_input.reshape((1, n_seq, 1, n_steps2, n_features))

In [ ]:
# Predict
yhat_log_scaled = model.predict(x_input, verbose=0)

In [ ]:
# Invert scaler
yhat_log = scaler_log_iq.inverse_transform(yhat_log_scaled)

In [ ]:
# Invert log
yhat = np.expm1(yhat_log)

In [ ]:
# Clean 
clean = combine_actual_and_LSTM_forecast_into_single_dataframe(this_run_test_original, yhat)

In [ ]:
# Graph
my_func.graph_actual_and_forecast_from_test(clean, str.title(this_run_city))

In [ ]:
# Score
# my_func.score(this_run_approach, this_run_details, this_run_city, this_run_data_set, 
#               clean['actual'], clean['forecast'], score_df)

In [ ]:
# Score
my_func.score(this_run_approach, this_run_variation, 
              this_run_details, this_run_city, this_run_data_set, this_run_transform, 
              clean['actual'], clean['forecast'], score_df)

In [ ]:
# Review score
score_df.tail(1)

## Summarize - all models - San Juan

In [ ]:
summary = score_df[(score_df['data'] == 'test') & (score_df['city'] == 'san juan')] 
summary.sort_values(by=['city', 'mae'], ascending=[False, True])

## Summarize - all models - Iquitos

In [ ]:
summary = score_df[(score_df['data'] == 'test') & (score_df['city'] == 'iquitos')] 
summary.sort_values(by=['city', 'mae'], ascending=[False, True])

## Save scores

In [ ]:
score_df.to_pickle('../sb_cap2_nb-99_data/scores_after_nbk_9.pickle')

In [ ]:
score_df.to_csv('../sb_cap2_nb-99_data/scores_after_nbk_9.csv', index=False)

## Commentary

In this notebook, I focus on long short-term memory models (LSTM).  LSTM are deep-learning models based on recurrent neural network (RNN) architectures.  LSTM networks have been used successfully to forecast time series because LTSM learn temporal dependencies.  LSTM accomplish this by their use of cells, input gates, output gates and forget gates.  This architecture allows the LSTM to read time series one time step at a time and build representations to be used in prediction.

There are a variety of LSTM.  In this notebook, I focus on simple LSTM, stacked LSTM, bidirectional LSTM, CNN-LSTM and ConvLSTM.

These models were a bit finicky, I found.  I need to:  get the multi-step input sequences in just the right order (i.e., samples of 100 X's and 50 y's), get the data formatting just right (i.e., lists v. arrays v. dataframes), and shape the data into the just right dimensions (ie., from 1D time series into 3D, 4D and 5D arrays).  But this is standard practice, of course.  For me, some of the pre-packaged functions to help with this (i.e., TimeSeriesGenerator) weren't easily adaptable to my multi-step time series problem.

**Simple LSTM**

* This model has one hidden layer of LSTM units and one output layer.
* I use 100 units as the default for all my LSTM models.
* My simple LSTM model is the baseline with an MAE of around 25.

**Stacked LSTM**

* Stacked models use multiple layers of hidden LSTM.
* My stacked LSTM model uses such layers.  This model improved my MAE to a bit below 25.
* Increasing the number of unit per LSTM node from 100 to 250 did not improve the model's performance. So, I continue to use 100 units in my later models.

**Bidirectional LSTM**

* In a bidirectional models, the LSTM learns the sequence both backward and forward.
* My bidirectional LSTM did worse than my baseline simple LSTM.

**CNN-LSTM**

* These are hybrid model that first process data through a convolutional neural network (CNN) and then through a simple LSTM.  These added CNN layers are to improve the model's ability to learn key one-dimensional sequences.
* My CNN-LSTM outperforms all prior LSTM models with an MAE of around 23.

**ConvLSTM**

* ConvLSTM are hybrid models, too.  They are much like CNN-LSTM.  However, the CNN-related layer is build into the LSTM layer, rather than being before the LSTM layer.
* My initial ConvLSTM has the best performance so far with an MAE of around near 22.
* To further improve perofrmance, I model off log-transformed data.  The score improved again with an MAE of around 21.
* The ConvLSTM on log-transformed data is the best of my LSTM models.

After using the San Juan data to select the model, I run the best-performing model on the Iquitos data.

Overall, the LSTM models do not perform very well.  ConvLSTM, the best of the LSTM, performs worse than the best of the Facebook Prophet, exponential smoothing and ARMIA models.